# read file contains All proteins to start pre processing

In [5]:
## All libraries needed for this step
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import umap
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
import scipy.stats as stats
from sklearn.decomposition import PCA
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram


In [ ]:
print("Current Working Directory:", os.getcwd())

In [ ]:

# file path 
file_path = "./total_protein_1.csv"

# Load dataset
data = pd.read_csv(file_path)

# Print success message and show the first few rows
print(data.head())

print("-" * 100)


# total number of rows and columns
rows, columns = data.shape

print(f"Rows: {rows}, Columns: {columns}")

print("-" * 100)

# Select columns from column 7 to column 33 (group for samples)
selected_columns = data.iloc[:, 6:33]  

# Get the number of selected columns
total_selected_columns = selected_columns.shape[1]

print(f"Total selected columns: {total_selected_columns}")
print(f"Selected column names: {list(selected_columns.columns)}")

print("-" * 100)

# Extract columns that represent samples using a regular expression
sample_columns = [col for col in data.columns if re.match(r'^(WtSham|WtLPS|WtLPSTB|WtShamTB|BtgSham|BtgLPS|BKSham|BKLPS)_\d+', col)]

# Count total number of samples
total_samples = len(sample_columns)

# Count the number of repeats for each group
group_counts = pd.Series([re.match(r'^[a-zA-Z]+', col).group(0) for col in sample_columns]).value_counts()

# Print results
print(f"Total samples: {total_samples}")
print("Repeats per group:")
print(group_counts)







#Check FDR distribution

In [ ]:
# calculate the percentage of proteins with High, Medium, and Low FDR confidence levels.

# Count the number of occurrences for each FDR confidence level
fdr_counts = data['Protein.FDR.Confidence..Combined'].value_counts()

# Calculate the total number of entries
total_proteins = fdr_counts.sum()

# Calculate the percentage for each FDR confidence level
fdr_percentage = (fdr_counts / total_proteins) * 100

# Display the results
fdr_percentage


In [ ]:
# Relative path to the preprocess folder
folder_path = "./"
file_name = "FDR_Distribution_1.pdf"

# Ensure the folder exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Count proteins by FDR confidence levels
fdr_counts = data['Protein.FDR.Confidence..Combined'].value_counts()

# Labels and sizes for the pie chart
labels = fdr_counts.index
sizes = fdr_counts.values
colors = ["#6667ff", "#66b3ff", "#66ffff"]

# Create a pie chart with a center circle (donut chart)
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors,
        wedgeprops=dict(width=0.6), pctdistance=0.89, 
        textprops={'fontsize': 12, 'fontweight': 'bold'})  # Adjusted font size for labels

# Draw a smaller circle at the center to make it look like a donut
centre_circle = plt.Circle((0, 0), 0.30, fc='white')  # Adjusted size of the center circle
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Set the title with the specified font size and bold styling
plt.title('', fontsize=14, fontweight='bold')

# Display the plot
plt.axis('equal')  # Equal aspect ratio ensures the pie is drawn as a circle

# Save the plot as a PDF
plt.savefig(file_name, format="pdf", bbox_inches="tight")

# Show the plot
plt.show()



In [ ]:
# Calculate the percentage of missing values in the entire dataset
missing_values_percent = data.isnull().mean().mean() * 100

# Print the percentage of missing values
print(f"Percentage of missing values in the dataset: {missing_values_percent:.2f}%")

## Filter Data based on -> ([FDR] == "High") & [PSMs] >= 2) & [Coverage] >= 20) criteria

In [ ]:
# Filtering the data for FDR == "high", PSMs >= 2, and coverage >= 20
folder_path = "./"

FDR="Protein.FDR.Confidence..Combined"

filtered_df = data[(data[FDR] == "High") & (data['PSMs'] >= 2) & (data['Coverage'] >= 20)]

# Specify the full file path for the output CSV file
output_path = "./filtered_FDR.csv"

# Save the filtered DataFrame to the specified path
filtered_df.to_csv(output_path, index=False)

print(f"Filtered data saved to: {output_path}")

In [ ]:
# Calculate the percentage of missing values in the entire dataset
missing_values_percent = filtered_df.isnull().mean().mean() * 100

# Print the percentage of missing values
print(f"Percentage of missing values in the dataset: {missing_values_percent:.2f}%")


In [17]:
### missingno and check missing values

In [ ]:
folder_path = "./"
matrix_file = "missing_matrix_1.pdf"
bar_file = "missing_bar_1.pdf"

# Select columns 7 to 33 (Python indexing is 0-based, and iloc excludes the last index)
selected_columns = filtered_df.iloc[:, 6:33]

# Plot 1: Missing values matrix
msno.matrix(selected_columns)
matrix_output_path = folder_path + matrix_file
plt.savefig(matrix_output_path, format="pdf", bbox_inches="tight")
plt.tight_layout()
plt.show()
print(f"Missing value matrix saved as: {matrix_output_path}")

# Plot 2: Missing values bar chart
msno.bar(selected_columns)
bar_output_path = folder_path + bar_file
plt.savefig(bar_output_path, format="pdf", bbox_inches="tight")
plt.tight_layout()
plt.show()
print(f"Missing value bar chart saved as: {bar_output_path}")


In [19]:
## speprate Check for Data Availability : at least 2/3 of replicates per groups 

In [ ]:
# Step 2: Re-define the groups and replicates
groups = {
    'WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'BtgLPS': ['BtgLPS5_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Step 3: Create a function to check if more than 66% of data is missing in each group
def calculate_missing_percentage(row, groups):
    for group, columns in groups.items():
        valid_columns = [col for col in columns if col in row.index]
        if valid_columns:
            missing_percentage = row[valid_columns].isnull().mean() * 100
            if missing_percentage > 66:
                return True  # This row has more than 66% missing values in at least one group
    return False

# Step 4: Apply the function to classify each row
filtered_df['more_than_66_missing'] = filtered_df.apply(calculate_missing_percentage, axis=1, groups=groups)

# Step 5: Split the data into two groups
more_than_66_df = filtered_df[filtered_df['more_than_66_missing']]
less_than_or_equal_66_df = filtered_df[~filtered_df['more_than_66_missing']]

# Step 6: Save both datasets as separate CSV files
more_than_66_file = "./more_than_66.csv"
less_than_or_equal_66_file = "./less_than_or_equal_66.csv"

more_than_66_df.to_csv(more_than_66_file, index=False)
less_than_or_equal_66_df.to_csv(less_than_or_equal_66_file, index=False)

(more_than_66_file, less_than_or_equal_66_file, len(more_than_66_df), len(less_than_or_equal_66_df))


In [21]:
## imputing missing data in data_less_66 by KNN techniques and check After imputation 

In [ ]:
# Step 1: Load the `less_than_or_equal_66.csv` file
file_path_2 = "./less_than_or_equal_66.csv"
data_less_66 = pd.read_csv(file_path_2)

# Define the 8 groups of columns
group_definitions = {
    'Group_1_WtSham': [ 'WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Function to apply KNN Imputation group-wise
def knn_impute_groupwise(data_less_66, group_definitions, n_neighbors=3):
    # Create a copy of the original data
    imputed_data = data_less_66.copy()
    
    # Loop over each group
    for group_name, group_columns in group_definitions.items():
        print(f"Imputing group: {group_name}")
        
        # Select the data corresponding to the current group
        group_data = data[group_columns]
        
        # Apply KNN Imputation to this group
        knn_imputer = KNNImputer(n_neighbors=n_neighbors)
        imputed_group_data = pd.DataFrame(knn_imputer.fit_transform(group_data), columns=group_columns)
        
        # Replace the imputed data back into the original dataframe
        imputed_data[group_columns] = imputed_group_data
    
    return imputed_data

# Apply KNN imputation group-wise
imputed_data = knn_impute_groupwise(data_less_66, group_definitions)

# Save the imputed data to a CSV file (optional)
output_file_path = "./imputed_data.csv"
imputed_data.to_csv(output_file_path, index=False)

print(f"Imputation completed and saved to {output_file_path}")


In [ ]:

# Load the dataset 
file_path = "./less_than_or_equal_66.csv"
data = pd.read_csv(file_path)

# Define the 8 groups of columns
group_definitions = {
    'Group_1_WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Function to apply KNN Imputation group-wise
def knn_impute_groupwise(data, group_definitions, n_neighbors=3):
    # Create a copy of the original data
    imputed_data = data.copy()
    
    # Loop over each group
    for group_name, group_columns in group_definitions.items():
        print(f"Imputing group: {group_name}")
        
        # Check if all columns in the group exist in the data
        missing_columns = [col for col in group_columns if col not in data.columns]
        if missing_columns:
            print(f"Skipping {group_name} due to missing columns: {missing_columns}")
            continue
        
        # Select the data corresponding to the current group
        group_data = data[group_columns]
        
        # Apply KNN Imputation to this group
        knn_imputer = KNNImputer(n_neighbors=n_neighbors)
        imputed_group_data = pd.DataFrame(knn_imputer.fit_transform(group_data), columns=group_columns)
        
        # Replace the imputed data back into the original dataframe
        imputed_data[group_columns] = imputed_group_data
    
    return imputed_data

# Apply KNN imputation group-wise
imputed_data = knn_impute_groupwise(data, group_definitions)

# Save the imputed data to a CSV file
output_file_path = "./imputed_data.csv"
imputed_data.to_csv(output_file_path, index=False)

print(f"Imputation completed and saved to {output_file_path}")


In [24]:
## check imputations

In [ ]:

# Load the datasets
data_before = pd.read_csv("./less_than_or_equal_66.csv")
data_after = pd.read_csv("./imputed_data.csv")

# Select only columns 8 to 34 (index 6 to 33)
data_before_selected = data_before.iloc[:, 6:33]
data_after_selected = data_after.iloc[:, 6:33]

# Count the missing values before and after imputation
missing_values_before = data_before_selected.isnull().sum().sum()
missing_values_after = data_after_selected.isnull().sum().sum()

# Create a bar plot to compare missing values before and after imputation
labels = ['Before Imputation', 'After Imputation']
values = [missing_values_before, missing_values_after]

plt.figure(figsize=(8, 5))
plt.bar(labels, values, color=['#ff6666', '#66b3ff'])
plt.title('Missing Values Before and After Imputation', fontsize=14, fontweight='bold')
plt.ylabel('Number of Missing Values', fontsize=12, fontweight='bold')
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig("./missing_values_comparison.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Create heatmaps to visualize missing values
# Heatmap BEFORE imputation
plt.figure(figsize=(12, 6))
sns.heatmap(data_before_selected.isnull(), cmap='viridis', cbar=False, yticklabels=False)
plt.title('Missing Values Before Imputation')
plt.tight_layout()
plt.savefig("./before_impute_heatmap.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Heatmap AFTER imputation
plt.figure(figsize=(12, 6))
sns.heatmap(data_after_selected.isnull(), cmap='viridis', cbar=False, yticklabels=False)
plt.title('Missing Values After Imputation')
plt.tight_layout()
plt.savefig("./after_impute_heatmap.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Print the results
print(f"Total missing values before imputation: {missing_values_before}")
print(f"Total missing values after imputation: {missing_values_after}")


In [ ]:
# Load the dataset before and after imputation using relative paths
file_path_before = "./less_than_or_equal_66.csv"
file_path_after = "./imputed_data.csv"
before_imputation_data = pd.read_csv(file_path_before)
after_imputation_data = pd.read_csv(file_path_after)

# Step 1: Select only columns 7 to 34 
numeric_before_imputation = before_imputation_data.iloc[:, 6:33]
numeric_after_imputation = after_imputation_data.iloc[:, 6:33]

# Step 2: Select only non-missing values from the 'before' dataset
non_missing_before_imputation = numeric_before_imputation.dropna()

# Step 3: Apply log2 transformation (with a small constant to avoid log(0))
log2_non_missing_before = np.log2(non_missing_before_imputation + 1e-9)
log2_after_imputation = np.log2(numeric_after_imputation + 1e-9)

# Step 4: Create visualizations to compare the distributions

# a) Histogram + KDE Comparison
plt.figure(figsize=(6, 4))

# Plot histogram of non-missing values before imputation
sns.histplot(log2_non_missing_before.stack(), bins=50, kde=True, color='blue', alpha=0.5, label='Before Imputation (Non-missing)')

# Plot histogram of all values after imputation
sns.histplot(log2_after_imputation.stack(), bins=50, kde=True, color='green', alpha=0.5, label='After Imputation (All Data)')

# Enhancements for font size and weight
plt.title('Histogram and KDE Comparison of Data Before and After Imputation (Log2 Transformed)', fontsize=12, fontweight='bold')
plt.xlabel('Log2 Values', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=12, fontweight='bold')
plt.legend(loc='upper right', fontsize=12, frameon=False)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Set tick labels to be bold and font size 12
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Save the plot as a PDF in the current directory
plt.tight_layout()
plt.savefig("./before_and_after_impute_dis.pdf", format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
# Load the dataset
file_path = "./imputed_data.csv"
data = pd.read_csv(file_path)

# Step 1: Select only columns 7 to 34
data_selected = data.iloc[:, 6:33]

# Step 2: Apply log2 transformation (with a small constant to avoid log(0))
log2_transformed_data = np.log2(data_selected + 1e-9)

# Step 3a: Box plot for log2 transformed data (before normalization)
plt.figure(figsize=(12, 6))
sns.boxplot(data=log2_transformed_data)
plt.title('Box Plot of Log2 Transformed Data (Before Normalization)', fontsize=14, fontweight='bold')
plt.xticks(rotation=90, fontsize=12)
plt.tight_layout()
plt.savefig("./before_normalization.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Simulate median normalization (replace with your actual normalization method)
median_normalized_data = log2_transformed_data.subtract(log2_transformed_data.median(axis=0), axis=1)

# Step 3b: Box plot for median-normalized data (after normalization)
plt.figure(figsize=(12, 6))
sns.boxplot(data=median_normalized_data)
plt.title('Box Plot of Median Normalized Data (After Normalization)', fontsize=14, fontweight='bold')
plt.xticks(rotation=90, fontsize=12)
plt.tight_layout()
plt.savefig("./after_normalization.pdf", format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
# Load the dataset
file_path = "./imputed_data.csv"
data = pd.read_csv(file_path)

# Step 1: Select only columns 8 to 27 for log2 transformation and normalization
data_selected = data.iloc[:, 6:33]

# Step 2: Apply log2 transformation (with a small constant to avoid log(0))
log2_transformed_data = np.log2(data_selected + 1e-9)

# Step 3: Perform median normalization
median_normalized_data = log2_transformed_data.subtract(log2_transformed_data.median(axis=0), axis=1)

# Step 4: Replace the original columns (8 to 27) with the normalized data
data.iloc[:, 6:33] = median_normalized_data

# Step 5: Remove the last column from the dataset
data = data.iloc[:, :-1]

# Step 6: Save the full dataset (with normalized columns and without the last column) as Final_Normalized.csv
data.to_csv("./Final_Normalized.csv", index=False)
print("Final normalized data has been saved as 'Final_Normalized.csv'.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Load the dataset
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)

# Extract columns 7 to 34 (Python index 6 to 33)
data_columns_7_34 = data.iloc[:, 6:33]

# 1. Check column medians
medians = data_columns_7_34.median()
print("Column Medians:")
print(medians)

# 2. Generate histograms
plt.figure(figsize=(15, 10))
for i, column in enumerate(data_columns_7_34.columns):
    plt.subplot(5, 6, i + 1)
    plt.hist(data_columns_7_34[column], bins=30, alpha=0.7, edgecolor='k')
    plt.title(column, fontsize=8)
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)

plt.tight_layout()
histogram_pdf_filename = "histograms_output.pdf"
plt.savefig(histogram_pdf_filename, format="pdf")
plt.close()
print(f"Histograms saved as {histogram_pdf_filename}")

# 3. Generate Q-Q plots
num_columns = len(data_columns_7_34.columns)
num_rows = (num_columns // 5) + 1

plt.figure(figsize=(20, 15))
for i, column in enumerate(data_columns_7_34.columns):
    plt.subplot(num_rows, 5, i + 1)
    stats.probplot(data_columns_7_34[column].dropna(), dist="norm", plot=plt)
    plt.title(column, fontsize=8)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
qq_plot_pdf_filename = "qq_plots_output.pdf"
plt.savefig(qq_plot_pdf_filename, format="pdf")
plt.close()
print(f"Q-Q Plots saved as {qq_plot_pdf_filename}")


In [30]:
## check normality of data 

In [ ]:
# Reload the uploaded file for accuracy
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)

# Extract columns 7 to 34 (Python index 6 to 33) for the check
data_columns_7_34_corrected = data.iloc[:, 6:33]

# 1. Check column medians to ensure they are close to zero
medians = data_columns_7_34_corrected.median()

# 2. Visualize histograms for all columns to check distribution
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
for i, column in enumerate(data_columns_7_34_corrected.columns):
    plt.subplot(5, 6, i + 1)
    plt.hist(data_columns_7_34_corrected[column], bins=30, alpha=0.7, edgecolor='k')
    plt.title(column)
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
plt.tight_layout()
plt.show()

# Save the figure as a PDF
pdf_filename = "histograms_output.pdf"
plt.savefig(pdf_filename, format="pdf")
plt.show()

print(f"Histograms saved as {pdf_filename}")


In [ ]:

# Extract columns 7 to 34 (Python index 6 to 33) for the check
data_columns_final_normalized = data.iloc[:, 6:33]

# Generate Q-Q plots for all columns in data_columns_final_normalized
num_columns = len(data_columns_final_normalized.columns)
num_rows = (num_columns // 5) + 1

plt.figure(figsize=(20, 15))
for i, column in enumerate(data_columns_final_normalized.columns):
    plt.subplot(num_rows, 5, i + 1)
    stats.probplot(data_columns_final_normalized[column].dropna(), dist="norm", plot=plt)
    plt.title(column, fontsize=8)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.show()


In [33]:
### shapiro _wilk normality test for each sample normality

In [ ]:
# Re-load the dataset since the environment reset
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)

# Re-define groups
groups = {
    'Group_1_WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Function to check normality for each sample in each group using Shapiro-Wilk test
from scipy.stats import shapiro

normality_results = {}

for group_name, samples in groups.items():
    normality_results[group_name] = {}
    for sample in samples:
        stat, p_value = shapiro(data[sample].dropna())
        normality_results[group_name][sample] = {'W-statistic': stat, 'p-value': p_value}

# Convert the results to a DataFrame for easier interpretation
normality_df = pd.DataFrame.from_dict({(i, j): normality_results[i][j]
                                       for i in normality_results.keys()
                                       for j in normality_results[i].keys()},
                                      orient='index')


# Display the first few rows of the results
print(normality_df)

In [ ]:


# Load your data (assuming it's already loaded as `data`)

# Step 1: Extract and scale the data (already log2-transformed and median-subtracted)
data_columns_final_normalized = data.iloc[:, 6:33]  # Select appropriate columns for analysis
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_columns_final_normalized.T)

# Step 2: Apply UMAP
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='euclidean', random_state=42)
umap_result = reducer.fit_transform(scaled_data)

# Step 3: Prepare group labels (ensure it matches the number of samples)
group_labels = [
    'Group_1', 'Group_1', 'Group_1',   # Group 1 - WtSham
    'Group_2', 'Group_2', 'Group_2', 'Group_2',  # Group 2 - WtLPS 
    'Group_3', 'Group_3', 'Group_3', 'Group_3',  # Group 3 - WtLPSTB
    'Group_4', 'Group_4', 'Group_4',   # Group 4 - WtShamTB
    'Group_5', 'Group_5', 'Group_5',   # Group 5 - BtgSham
    'Group_6', 'Group_6', 'Group_6', 'Group_6',  # Group 6 - BtgLPS
    'Group_7', 'Group_7', 'Group_7',   # Group 7 - BKSham
    'Group_8', 'Group_8', 'Group_8'    # Group 8 - BKLPS
]

# Ensure group_labels length matches the number of rows in umap_result
if len(group_labels) != umap_result.shape[0]:
    raise ValueError(f"Length of group_labels ({len(group_labels)}) does not match the number of samples ({umap_result.shape[0]}).")

# Step 4: Convert UMAP results into a DataFrame
umap_df = pd.DataFrame(umap_result, columns=['UMAP1', 'UMAP2'])
umap_df['Group'] = group_labels

# Step 5: Plot UMAP results
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x='UMAP1', y='UMAP2', hue='Group', data=umap_df, 
    palette='Set2', s=100, alpha=0.8, edgecolor='k'
)
plt.title('UMAP for Batch Effect Detection')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Group")
plt.grid(True)


# Save the UMAP plot as a PDF
umap_pdf_filename = "umap_batch_effect.pdf"
plt.savefig(umap_pdf_filename, format="pdf")
plt.show()
print(f"UMAP plot saved as {umap_pdf_filename}")


In [ ]:


# Re-create scaled data as it might have been cleared from memory
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_columns_final_normalized.T)

# Step 1: Apply PCA to the scaled data
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Step 2: Convert PCA results into a DataFrame
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['Group'] = group_labels

# Step 3: Plot the PCA results
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='PC1', y='PC2', hue='Group', data=pca_df, 
    palette='Set2', s=100, alpha=0.8, edgecolor='k'
)
plt.title('PCA for Batch Effect Detection')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}% variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}% variance)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Group")
plt.grid(True)

# Save the UMAP plot as a PDF
umap_pdf_filename = "umap_batch_effect_2.pdf"
plt.savefig(umap_pdf_filename, format="pdf")
plt.show()
print(f"UMAP plot saved as {umap_pdf_filename}")



In [ ]:


# Load your data (assuming it's already loaded as `data`)
data_columns_final_normalized = data.iloc[:, 6:33]  # Adjust the column range if needed

# Step 1: Scale the data (already log2-transformed and median-subtracted)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_columns_final_normalized.T)

# Step 2: Apply UMAP
reducer = umap.UMAP(n_neighbors=6, min_dist=0.1, metric='euclidean', random_state=42)
umap_result = reducer.fit_transform(scaled_data)

# Step 3: Prepare group labels with descriptive names
group_labels = [
    'WtSham', 'WtSham', 'WtSham',
    'WtLPS', 'WtLPS', 'WtLPS', 'WtLPS',
    'WtLPSTB', 'WtLPSTB', 'WtLPSTB', 'WtLPSTB',
    'WtShamTB', 'WtShamTB', 'WtShamTB',
    'BtgSham', 'BtgSham', 'BtgSham',
    'BtgLPS', 'BtgLPS', 'BtgLPS', 'BtgLPS',
    'BKSham', 'BKSham', 'BKSham',
    'BKLPS', 'BKLPS', 'BKLPS'
]

# Ensure group_labels length matches the number of rows in umap_result
if len(group_labels) != umap_result.shape[0]:
    raise ValueError(f"Length of group_labels ({len(group_labels)}) does not match the number of samples ({umap_result.shape[0]}).")

# Step 4: Convert UMAP results into a DataFrame
umap_df = pd.DataFrame(umap_result, columns=['UMAP1', 'UMAP2'])
umap_df['Group'] = group_labels

# Save the UMAP plot as a PDF


# Step 5: Plot UMAP with sample group names as labels
plt.figure(figsize=(6, 4))
sns.scatterplot(
    x='UMAP1', y='UMAP2', hue='Group', data=umap_df, 
    palette='Set2', s=150, alpha=0.8, edgecolor='k')

plt.title('UMAP for Batch Effect Detection (with Group Names)')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Group")
plt.grid(True)



# Save the figure with a larger page size and tight bounding box
umap_pdf_filename = "umap_batch_effect_large.pdf"
plt.savefig(umap_pdf_filename, format="pdf", bbox_inches="tight", dpi=300)
plt.close()

print(f"UMAP plot saved as {umap_pdf_filename}")

plt.show()
print(f"UMAP plot saved as {umap_pdf_filename}")



In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Load the dataset
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)

# Define the sample groups
groups = {
    'Group_1_WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Melt the data to long format for ANOVA
sample_columns = [col for group in groups.values() for col in group]
melted_data = data[sample_columns].melt(var_name="Sample", value_name="Expression")

# Create a batch variable based on the group each sample belongs to
batch_mapping = {sample: group for group, samples in groups.items() for sample in samples}
melted_data["Batch"] = melted_data["Sample"].map(batch_mapping)

# Step 2: Perform ANOVA to test for batch effects
model = ols("Expression ~ C(Batch)", data=melted_data).fit()
anova_results = anova_lm(model)






# Display ANOVA results
anova_results


In [39]:
## the P value > 0.05 and batch effect is not statisticaly significant. NO batch effect in data .

In [ ]:

# Load the data
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)


# Extract only the sample columns for PCA analysis
sample_columns = [
    'WtSham_2', 'WtSham_3', 'WtSham_4', 'WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4', 
    'WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4', 'WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3', 
    'BtgSham_1', 'BtgSham_2', 'BtgSham_3', 'BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4', 
    'BKSham_1', 'BKSham_2', 'BKSham_3', 'BKLPS_1', 'BKLPS_2', 'BKLPS_3'
]

# Prepare the data for PCA by dropping rows with any missing values in these columns
data_pca = data[sample_columns].dropna()

# Apply PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_pca.T)

# Create a DataFrame for the PCA result
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['Sample'] = sample_columns

# Generate a heatmap of the sample correlations to check for batch effects
plt.figure(figsize=(15, 10))
sns.heatmap(data_pca.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap of Sample Correlation')
plt.show()




In [ ]:


# Load the data
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)

# Extract the relevant columns (all columns representing samples)
sample_columns = data.columns[6:33]  # Adjust based on the actual columns containing sample data

# Transpose for distance calculations
data_matrix = data[sample_columns].T.to_numpy()

# Step 1: Calculate Euclidean distances for all columns
euclidean_distances = pdist(data_matrix, metric='euclidean')
euclidean_distances_matrix = squareform(euclidean_distances)

# Step 2: Plot the Euclidean distance heatmap for all columns
plt.figure(figsize=(15, 12))
sns.heatmap(euclidean_distances_matrix, annot=True, xticklabels=sample_columns, yticklabels=sample_columns, cmap="YlGnBu")
plt.title("Euclidean Distances Between All Samples")
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Save the plot as a PDF
output_path = "./Euclidean_Distances_All_Samples.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')

# Show the plot
plt.show()

# Provide the location of the saved PDF
print(f"The plot has been saved at: {output_path}")


In [ ]:

# Load the data
file_path = "./Final_Normalized.csv"
data = pd.read_csv(file_path)

#Define groups based on available sample columns
# Define the 8 groups of columns
groups= {
    'Group_1_WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Compute group means
group_means = {}
for group, samples in groups.items():
    group_means[group] = data[samples].mean(axis=1)

# Convert to DataFrame
group_means_df = pd.DataFrame(group_means)

# Transpose for distance calculations
group_means_matrix = group_means_df.T.to_numpy()

# Calculate Euclidean distances
euclidean_distances = pdist(group_means_matrix, metric='euclidean')
euclidean_distances_matrix = squareform(euclidean_distances)

# Plot Euclidean distances heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(euclidean_distances_matrix, annot=True, xticklabels=group_means_df.columns, yticklabels=group_means_df.columns, cmap="YlGnBu")
plt.title("Euclidean Distances Between Groups")


# Save the plot as a PDF
output_path = "./Eucleadian distance.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')

# Show the plot
plt.show()

# Provide the location of the saved PDF
output_path


plt.show()




In [ ]:

# Define the groups based on available sample columns
# Define the 8 groups of columns
groups = {
    'Group_1_WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Check that the groups match the column names in the DataFrame
for group, samples in groups.items():
    missing_cols = [col for col in samples if col not in data.columns]
    if missing_cols:
        print(f"Warning: The following columns are missing for group {group}: {missing_cols}")

# Compute group means (average intensity across samples for each group)
group_means = {}
for group, samples in groups.items():
    group_means[group] = data[samples].mean(axis=1)

# Convert to DataFrame for easier handling
group_means_df = pd.DataFrame(group_means)

# Transpose the group means matrix for distance calculations
group_means_matrix = group_means_df.T.to_numpy()

# Calculate Euclidean distances between group means
euclidean_distances = pdist(group_means_matrix, metric='euclidean')

# Perform hierarchical clustering using linkage
linkage_matrix = linkage(euclidean_distances, method='average')

# Plot the dendrogram
plt.figure(figsize=(12, 8))
dendrogram(linkage_matrix, labels=group_means_df.columns, orientation='top', leaf_rotation=90)
plt.title('Dendrogram of Group Similarities (Euclidean Distance)')
plt.xlabel('Groups')
plt.ylabel('Euclidean Distance')
plt.tight_layout()

# Save the plot as a PDF (in the current working directory for now)
output_path_dendrogram = "Dendrogram_Euclidean_distance.pdf"
plt.savefig(output_path_dendrogram, format='pdf', bbox_inches='tight')

# Show the plot
plt.show()

# Print the location of the saved PDF
print(f"Dendrogram plot saved at: {output_path_dendrogram}")

In [ ]:

# Define the 8 groups of columns
groups = {
    'Group_1_WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'Group_2_WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'Group_3_WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'Group_4_WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'Group_5_BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'Group_6_BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'Group_7_BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'Group_8_BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# List to store mean values for each group
group_means = {}

# Loop through each group and calculate the mean for each protein
for group_name, group_columns in group_definitions.items():
    group_means[group_name] = data[group_columns].mean(axis=1)

# Convert the group means dictionary into a DataFrame for easier handling
group_means_df = pd.DataFrame(group_means)

# Create a clustermap with hierarchical clustering
plt.figure(figsize=(14, 12))  # Increase the figure size for better spacing
clustermap=sns.clustermap(group_means_df, annot=False, cmap='coolwarm', cbar_kws={'label': 'Log2 Normalized Protein Abundance'},
               xticklabels=group_means_df.columns, yticklabels=False, method='complete', metric='euclidean',
               dendrogram_ratio=(0.1, 0.34), figsize=(14, 12), tree_kws={'linewidths': 2})  # Increased linewidth for tree

# Set title and labels for clarity
plt.title("", fontsize=15)
plt.xlabel(" ", fontsize=12)
plt.ylabel("Mean log2-normalized Abundance", fontsize=12,rotation=90)
plt.tight_layout()
plt.show()


# Save the clustermap as a high-resolution PDF
clustermap_pdf_filename = "protein_clustermap.pdf"
clustermap.fig.savefig(clustermap_pdf_filename, format="pdf", bbox_inches="tight", dpi=300)  # Use `.fig` to access figure

print(f"Clustermap saved as {clustermap_pdf_filename}")
-

